In [1]:
import plotly.graph_objects as go

In [2]:
import requests 
import pandas as pd

In [5]:
ticker ='MSFT'

def getfinance(ticker):
    r = requests.get(f'https://financialmodelingprep.com/api/v3/income-statement/{ticker}?period=quarter&apikey=76540e8276fea01b3fe196deac9404e4')
    r = r.json()
    count = 0
    #d = r[0]
    for item in r:
        if count == 0:
            d = r[0]
            income_initial = pd.DataFrame(list(d.items()), columns = ['Item', 'Value'])

        count = count + 1
        if count > 5:
            continue
        d = r[count]
        income_statement2 = pd.DataFrame(list(d.items()), columns = ['Item', 'Value'])
        income_statement = income_initial.merge(income_statement2, on = 'Item', how = 'left')


    income_statement.columns = income_statement.iloc[0]
    income_statement = income_statement[5:-2]
    cols = income_statement.columns
    cols  = cols[1:]
    income_statement[cols] = income_statement[cols].apply(pd.to_numeric, errors = 'coerce')
    return income_statement

income_statement = getfinance(ticker)

In [6]:
income_statement

,date,2020-06-30,2019-03-31
5,revenue,3.803300e+10,3.057100e+10
6,costOfRevenue,1.233900e+10,1.017000e+10
7,grossProfit,2.569400e+10,2.040100e+10
8,grossProfitRatio,6.755710e-01,6.673320e-01
9,researchAndDevelopmentExpenses,5.214000e+09,4.316000e+09
10,generalAndAdministrativeExpenses,7.073000e+09,5.744000e+09
11,sellingAndMarketingExpenses,5.417000e+09,4.565000e+09
12,otherExpenses,-6.200000e+07,0.000000e+00
13,operatingExpenses,1.230800e+10,1.006700e+10
14,costAndExpenses,2.464700e+10,2.023700e+10


In [7]:
Rev = income_statement[income_statement['date']=='revenue'].iloc[0][1]
COGS = income_statement[income_statement['date']=='costOfRevenue'].iloc[0][1] * -1
GrossProfit = income_statement[income_statement['date']=='grossProfit'].iloc[0][1]

RD = income_statement[income_statement['date']=='researchAndDevelopmentExpenses'].iloc[0][1] * -1

GA = income_statement[income_statement['date']=='generalAndAdministrativeExpenses'].iloc[0][1] * -1

OperExp = income_statement[income_statement['date']=='operatingExpenses'].iloc[0][1] * -1



Int = income_statement[income_statement['date']=='interestExpense'].iloc[0][1] *-1
EBT = income_statement[income_statement['date']=='incomeBeforeTax'].iloc[0][1]
IncTax = income_statement[income_statement['date']=='incomeTaxExpense'].iloc[0][1] *-1
NetInc = income_statement[income_statement['date']=='netIncome'].iloc[0][1]

In [10]:
import plotly.graph_objects as go

fig = go.Figure(go.Waterfall(
    name = "20", orientation = "v",
    measure = ["relative", "relative", "total", "relative", "relative", "total", "relative", 'total', 'relative','total' ],
    x = ["Revenue", "COGS", "Gross Profit", "R&D", "G&A", "OperExp", 'Interest Expense', 'EBT', 'Income Tax', 'Net Income'],
    textposition = "outside",
    text = [Rev/1000000, COGS/1000000, GrossProfit/1000000, RD/1000000, GA/1000000, OperExp/1000000,
           Int/1000000,EBT/1000000, IncTax/1000000,NetInc/1000000],
    y = [Rev, COGS, GrossProfit, RD, GA,OperExp,Int ,EBT, IncTax, NetInc],
    connector = {"line":{"color":"rgb(63, 63, 63)"}},
    increasing = {'marker':{'color': 'green'}},
    decreasing = {'marker':{'color': 'red'}},
    totals = {'marker':{'color': 'blue'}},
))

fig.update_layout(
        title = "Profit and loss from last available quarter",
        showlegend = True
)

fig.show()